In [176]:
from groq import Groq
import os
import pandas as pd
import numpy as np
from pydub import AudioSegment
from langchain.text_splitter import TokenTextSplitter
from langchain.docstore.document import Document
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from openai import OpenAI


In [198]:
client = Groq(api_key = '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')
model = 'whisper-large-v3'

In [191]:
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    # api_key="My API Key",
    api_key = 'sk-efqdWVOfsRqT3JwECMa0NG3bVoqQ5c4Kebeo_l_zYKT3BlbkFJNVMlXlnaSV_Jh0X-Fzu_YXMNmTu9Ow9enymnl3hl0A',
)

In [192]:
# def transcribe_audio(audio_file_path):
#     with open(audio_file_path, 'rb') as audio_file:
#         transcription = client.audio.transcriptions.create("whisper-1", audio_file)
#     return transcription['text']

#open ai transcribe
def transcribe_audio(audio_file_path):
    audio_file = open(audio_file_path, "rb")
    transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
    )
    return transcript.text

In [105]:
file = './Dallas County/Richardson Fire Aug 13/31471-1723525806-chunks/chunk_1.mp3' 
print(transcribe_audio(file))

Transcription(text='1-4-1 at position.')


In [46]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [72]:
def chunk_audio(file_path, output_dir, silence_thresh=-50, min_silence_len=1000, keep_silence=1000):
    """
    Chunk an audio file into segments where there is sound.

    Parameters:
    - file_path: Path to the input audio file (e.g., .mp3).
    - output_dir: Directory where the output chunks will be saved.
    - silence_thresh: Silence threshold in dBFS below which the audio is considered silence.
    - min_silence_len: Minimum length of silence (in ms) that will be used to detect splits.
    - keep_silence: Amount of silence (in ms) to keep at the beginning and end of each chunk.

    Output:
    - Saves chunks of the audio file to the specified output directory.
    """

    # Load the audio file
    audio = AudioSegment.from_file(file_path, format="mp3")

    # Split the audio based on silence
    chunks = split_on_silence(
        audio,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh,
        keep_silence=keep_silence
    )

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Export each chunk as a separate audio file
    for i, chunk in enumerate(chunks):
        chunk_file_path = os.path.join(output_dir, f"chunk_{i + 1}.mp3")
        chunk.export(chunk_file_path, format="mp3")
        print(f"Exported {chunk_file_path}")

In [51]:
# input_file = "/path/to/your/audiofile.mp3"
input_file = './Dallas County/Richardson Fire Aug 13/31471-1723583087.mp3' 
output_directory = "./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks"

chunk_audio(input_file, output_directory)

Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_1.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_2.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_3.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_4.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_5.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_6.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_7.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_8.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_9.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_10.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/chunk_11.mp3
Exported ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks/ch

In [73]:
#write function to chunk audio files from a directory using chunk_audio function above and save the chunks in a new directory having the same name as the audio file without the extension appended with -chunks

def chunk_audio_directory(input_dir, output_dir):
    """
    Chunk all audio files in a directory and save the chunks in a new directory.

    Parameters:
    - input_dir: Path to the directory containing the input audio files.
    - output_dir: Path to the directory where the output chunks will be saved.
    """

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all files in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith(".mp3"):
            # Construct the input and output file paths
            input_file = os.path.join(input_dir, file_name)
            output_directory = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}-chunks")

            # Chunk the audio file
            chunk_audio(input_file, output_directory)

# input_directory = "/path/to/your/input_directory"
# output_directory = "/path/to/your/output_directory"





In [92]:
from concurrent.futures import ThreadPoolExecutor

# Define the chunking function
def chunk_audio_file(file_name, input_dir, output_dir):
    input_file = os.path.join(input_dir, file_name)
    output_directory = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}-chunks")
    chunk_audio(input_file, output_directory)


def chunk_audio_directory_parallel(input_dir, output_dir, num_processes=4):
    """
    Chunk all audio files in a directory and save the chunks in a new directory using multiprocessing.

    Parameters:
    - input_dir: Path to the directory containing the input audio files.
    - output_dir: Path to the directory where the output chunks will be saved.
    - num_processes: Number of processes to use for multiprocessing.
    """

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get a list of all audio files in the input directory
    audio_files = [f for f in os.listdir(input_dir) if f.endswith(".mp3")]


    # Use multiprocessing to chunk the audio files in parallel
    with ThreadPoolExecutor(max_workers=num_processes) as executor:
        futures = [executor.submit(chunk_audio_file, f, input_dir, output_dir) for f in audio_files]
        for future in futures:
            future.result()  # Wait for all tasks to complete


In [93]:

input_directory = './Dallas County/Dallas City Fire and Rescue Aug 13'
output_directory = './Dallas County/Dallas City Fire and Rescue Aug 13'

chunk_audio_directory_parallel(input_directory, output_directory)

Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723542604-chunks/chunk_1.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723542604-chunks/chunk_2.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723558714-chunks/chunk_1.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723542604-chunks/chunk_3.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723558714-chunks/chunk_2.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723542604-chunks/chunk_4.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723558714-chunks/chunk_3.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723542604-chunks/chunk_5.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723558714-chunks/chunk_4.mp3
Exported ./Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723542604-chunks/chunk_6.mp3
Exported ./Dallas County/Dallas City Fire and Resc

In [75]:
input_directory = './Dallas County/Sachse Police and Fire Aug 13'
output_directory = './Dallas County/Sachse Police and Fire Aug 13'
chunk_audio_directory(input_directory, output_directory)

Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_1.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_2.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_3.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_4.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_5.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_6.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_7.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_8.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_9.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_10.mp3
Exported ./Dallas County/Sachse Police and Fire Aug 13/2386-1723611487-chunks/chunk_11.mp3
Exported

In [66]:
def audio_to_text(filepath):
    with open(filepath, "rb") as file:
        translation = client.audio.translations.create(
            file=(filepath, file.read()),
            model="whisper-large-v3",
        )
    return translation.text

In [194]:
#write a function to convert audio from directory ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks to text and stitch them together using function audio_to_text
import time 

def audio_to_text_chunks(filepath):

    # Get a list of all the audio files in the directory
    audio_files = [f for f in os.listdir(filepath) if f.endswith(".mp3")]

    # Sort the audio files based on their index
    audio_files.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

    # Initialize an empty string to store the transcribed text
    transcribed_text = ""

    # Loop through each audio file
    for audio_file in audio_files:
        # Get the full path of the audio file
        audio_file_path = os.path.join(filepath, audio_file)
        
        # Transcribe the audio file
        #delay to avoid rate limit of 40 requests per minute

        time.sleep(1.25)
        
        text = transcribe_audio(audio_file_path)

        # Append the transcribed text to the full text
        transcribed_text += text + " "

    return transcribed_text


In [195]:
#write a function to use audio_to_text_chunks to convert audio files from a directory to text and save the text in a text file with the same name as the directory in the same directory 

def audio_to_text_directory(input_dir, output_dir):
    """
    Convert all audio files in a directory to text and save the text in a text file.

    Parameters:
    - input_dir: Path to the directory containing the input audio files.
    - output_dir: Path to the directory where the output text files will be saved.
    """

    # Loop through all directories in the input directory
    for dir_name in os.listdir(input_dir):
        if not os.path.isdir(os.path.join(input_dir, dir_name)):
            continue
        # Construct the input and output directory paths
        input_directory = os.path.join(input_dir, dir_name)
        output_file = os.path.join(output_dir, f"{dir_name}.txt")

        # Convert the audio files in the directory to text
        text = audio_to_text_chunks(input_directory)

        # Save the transcribed text to a text file
        with open(output_file, "w") as file:
            file.write(text)

# input_directory = "/path/to/your/input_directory"
# output_directory = "/path/to/your/output_directory"
input_directory = './Dallas County/Sachse Police and Fire Aug 13'
output_directory = './Dallas County/Sachse Police and Fire Aug 13'
audio_to_text_directory(input_directory, output_directory)


# transcribed_text = audio_to_text_chunks('./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks')


In [70]:
#function to calculate length of chunked files in seconds from directory ./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks

def get_audio_lengths(filepath):
    # Get a list of all the audio files in the directory
    audio_files = [f for f in os.listdir(filepath) if f.endswith(".mp3")]

    # Initialize an empty list to store the lengths of the audio files
    audio_lengths = []

    # Loop through each audio file
    for audio_file in audio_files:
        # Get the full path of the audio file
        audio_file_path = os.path.join(filepath, audio_file)
        
        # Load the audio file
        audio = AudioSegment.from_file(audio_file_path, format="mp3")
        
        # Get the length of the audio file in seconds
        length_seconds = len(audio) / 1000
        
        # Append the length to the list
        audio_lengths.append(length_seconds)

    return audio_lengths

audio_lengths = get_audio_lengths('./Dallas County/Richardson Fire Aug 13/31471-1723583087-chunks')

#calculate sum in seconds of all the chunks
total_length = sum(audio_lengths)

In [69]:
print(transcribed_text)

 31, station.  31 at the station, 1603.  Engine 2 clear. No second, Andrew.  Be clear 1604.  Thank you.  Email is ready for you.  33 is going to be clear. No patient.  33 is clear at 1607.  And it's a reservation.  The station at 1608.  This is SDS from truck 4, truck 4 back in service on the air.  Truck 4 on the air, 16-16.  This is Ash from Engine 4. Engine 4 is on the air.  We're on the air at 1660.  Dispatch MS-34 is clear, on the air, back in service.  34 on the air 1315.  Engine two, station. Engine two, station, sixteen seventeen.  Ambulance 33, station.  Airmen, this is 33, this is Station at 1624.  This is Ashman, hit the board, hit the board, this is Major.  And your board decision is 1625.  Good question, Chuck Ford.  Go ahead.  Thank you.  Thank you.  This is Stratis Amateur 4 at the station.  AMOUNT 34 ATTESTATION 1629 


In [9]:
filepath = "2681-1723266942.mp3"
translation_text = audio_to_text(filepath)
print(translation_text)


 Thank you. Thank you. Thank you. Thank you. Melody Lane, we got a three-story apartment building pitched to the reef. Composition shingles still on CNA from the outside. Looks like 37 truck. Engine 37, we're making an investigation. Everybody continue emergency response. SNAID, you're on the middle of the lane. Battalion 4 is going to be in command. Nothing showing at this time. Engine 37 is investigating continued emergency response. Battalion 4 is in command at 12-16. Oh, drop. Okay. Box 37, double-op 51 at 12-17, truck 37, and engine 37 remain on scene. Engine 6. Engine 6, who did you want me to swap you with? Well, I'm saying I don't think there needs to be a swap. Just grab Engine 6. Thank you. And rather than Jay West, I have Forrest. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you Thank you Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you. Thank you Thank you

In [36]:
# def get_audio_files(path):
#     files = []
#     for r, d, f in os.walk(path):
#         for file in f:
#             if '.mp3' in file:
#                 files.append(os.path.join(r, file))
#     return files


#find all the folders in the directory, then find all the mp3 files in those folders and then create a dictionary with folder name as the key and the list of mp3 files as the value
def get_audio_files(path):
    files = {}
    for r, d, f in os.walk(path):
        for file in f:
            if '.mp3' in file:
                if r in files:
                    files[r].append(os.path.join(r, file))
                else:
                    files[r] = [os.path.join(r, file)]
    return files
# def get_audio_files(path):
#     files = []
#     for r, d, f in os.walk(path):
#         for file in f:
#             if '.mp3' in file:
#                 files.append(os.path.join(r, file))
#     return files

In [39]:
path = "./Dallas County"
# path = './Tarrant County'

files = get_audio_files(path)
print(files)

{'./Dallas County/Richardson Fire Aug 13': ['./Dallas County/Richardson Fire Aug 13/31471-1723583087.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723599197.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723579507.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723606357.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723566977.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723586667.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723608147.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723577717.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723561607.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723581297.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723543706.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723609937.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723575927.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723554447.mp3', './Dallas County/Richardson Fire Aug 13/31471-1723556237.mp3', './Dallas C

In [ ]:
#write code to loop loop dictionary and get the transcription of each audio file and store it in a dictionary with the folder name as the key and the transcription as the value and then write it to a csv file

transcriptions = {}

for folder, file_list in files.items():
    for file in file_list:
        transcription = transcribe_audio(file)
        transcriptions[file] = transcription
    #write it to a csv file
    df = pd.DataFrame(list(transcriptions.items()), columns=['file', 'transcription'])
    df.to_csv(f'{folder}.csv', index=False)
    


In [23]:
#write code to loop through all the files in the folder



def get_audio_translations(files):
    translations = []
    for file in files:
        translation = audio_to_text(file)
        translations.append(translation)
    return translations

#current folder 
path = "./"
files = get_audio_files(path)
translations = get_audio_translations(files[-2:])

#stich all the translations together
all_translations = " ".join(translations)


print(all_translations)

# #split the text into sentences
# splitter = TokenTextSplitter()
# sentences = splitter.split_sentences(all_translations)



 Thank you. Battalion 4 is on location with a single story, single family residence. Nothing showing from the outside, but we do have a waiver. Battalion 4 has command. First in-engine truck, command to investigate all other units level 1 stage. About 9344 Loma Vista Drive, single-story, one family residence. Let me tell you at this time, we do have waivers. First in-engine, first truck, go ahead and investigate all other units level 1 staging. Station 24 has command 1709. Station 28 out with the 24 seconds left in investigation mode. Station 28 is out in investigation mode 1710. 6 miles to 24, 15 miles to move to drop. 7's out, standing by a plug, 1710. Perfect, 7's out. 1710. Loma Vista Command. Okay. Loma Vista Command. You can disregard 2nd Chief, 2nd Truck, and Ritt. Loma Vista Command, Truck 37, Battalion 2, Engine 20, you may disregard per command. Battalion 2, you may disregard. Agents, will be received as required. Thank you. Sorry, honors. Sorry, I don't know if you can hear 

In [29]:
path = "./"
files = get_audio_files(path)
# len(files)
files[-2:]

['./audio/2681-1723327802.mp3', './audio/2681-1723342122.mp3']

In [169]:
def transcript_chat_completion(client, transcript, user_question):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": '''Given below is the transcript for different chunks of audio during 30 minute window of a fire station in the U.S. Please answer the questions citing the context.
                {transcript}
                '''.format(transcript=transcript)
            },
            {
                "role": "user",
                "content": user_question,
            }
        ],
        model="llama-3.1-70b-versatile",
        # response_format={ "type": "json_object" },
    )
    print(chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

# user_question = "List all the instances where the speaker mentions any address or related word to emphasize any address in the audio files. Provide the timestamp and the context of the mention. Always provide a few words before and after the address to give context of what the emergency might be. Provide exact words it speaks."


In [146]:
user_question = "List all the instances where the speaker mentions any address or related word to emphasize any address in the audio files. Provide the address and the context of the mention. Always provide a few words before and after the address to give context of what the emergency might be. Provide exact words it speaks. Give it in a json format with address and context as keys and the values as the address and context."

transcript_chat_completion(client, './Dallas County/Dallas City Fire and Rescue Aug 13/2681-1723581984-chunks.txt', user_question) 

However, I don't see the transcript of the audio file you mentioned. Please provide the content of the transcript file (`2681-1723581984-chunks.txt`) so I can assist you in extracting the addresses and their context.

Once I have the transcript, I'll provide the result in a JSON format like this:

```json
[
  {
    "address": "123 Main St",
    "context": "Dispatcher: Units respond to 123 Main St, reported structure fire. Engine 12, Engine 7, respond with caution."
  },
  {
    "address": "456 Elm St",
    "context": "Caller: Yeah, the fire is at 456 Elm St. It's a garage fire. Hurry!"
  },
  ...
]
```

Please paste the transcript, and I'll get started!


In [167]:
from pydantic import BaseModel

class AddressContext(BaseModel):
    address: str
    context: str

class AddressContextList(BaseModel):
    address_contexts: list[AddressContext]  

# Define the completion function
def transcript_chat_completion(client, transcript):
    completion = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": '''Given below is the transcript for different chunks of audio during 30 minute window of a fire station in the U.S. Please answer the questions citing the context.
                    {transcript}
                    '''.format(transcript=transcript)},
            {"role": "user", "content": "List all the instances where the speaker mentions any address or related word to emphasize any address in the audio files. Provide the address and the context of the mention. Always provide a few words before and after the address to give context of what the emergency might be. Provide exact words it speaks. Give it in a json format with address and context as keys and the values as the address and context."}
        ],
        response_format=AddressContextList,
    )

    math_reasoning = completion.choices[0].message.parsed
    return math_reasoning

In [162]:
#write chat completetion function using openai to get the answer to the question from the user

def chat_completion(client, user_question, model="gpt-4o-2024-08-06"):
    completion = client.completions.create(
        model=model,
        prompt=user_question,
        response_format={ "type": "json_object" },
    )
    return completion.choices[0].message.content

In [166]:
import csv 
#import pydantic model

user_question = "List all the instances where the speaker mentions any address or related word to emphasize any address in the audio files. Provide the address and the context of the mention. Always provide a few words before and after the address to give context of what the emergency might be. Provide exact words it speaks. Give it in a json format with address and context as keys and the values as the address and context."




def process_txt_files(input_directory, output_csv):
    # Prepare the CSV file for writing
    with open(output_csv, mode='w', newline='') as csvfile:
        fieldnames = ['address', 'context']
        # writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        

        writer.writeheader()

        # Loop through all .txt files in the directory
        for filename in os.listdir(input_directory):
            if filename.endswith(".txt"):
                file_path = os.path.join(input_directory, filename)

                # Read the content of the file
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()

                #account for the rate limit of 30 requests per minute
                time.sleep(2)
                try:
                    result = transcript_chat_completion(client, content, user_question)
                    print(result)

                    # Validate the result is a dictionary and has the required keys
                    if isinstance(result, dict) and 'address' in result and 'context' in result:
                        writer.writerow(result)
                    else:
                        raise ValueError(f"Invalid result format for file {filename}")

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

# input_directory = "/path/to/your/input_directory"
# output_csv = "/path/to/your/output_csv"
input_directory = './Dallas County/Dallas City Fire and Rescue Aug 13'
output_csv = './Dallas County/Dallas City Fire and Rescue Aug 13.csv'

process_txt_files(input_directory, output_csv)


Error processing file 2681-1723539024-chunks.txt: Object of type ModelMetaclass is not JSON serializable
Error processing file 2681-1723556924-chunks.txt: Object of type ModelMetaclass is not JSON serializable
Error processing file 2681-1723583774-chunks.txt: Object of type ModelMetaclass is not JSON serializable
Error processing file 2681-1723587354-chunks.txt: Object of type ModelMetaclass is not JSON serializable


KeyboardInterrupt: 

In [199]:
#browse through the transcriptions .txt files in the directory and pass it through transcript_chat_completion store the result [address, context ] in a row of a csv file with file name as the directory name 

def transcript_chat_completion(client, transcript, user_question):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": '''Given below is the transcript for different chunks of audio during 30 minute window of a fire station in the U.S. Please answer the questions citing the context. 
                {transcript}
                '''.format(transcript=transcript)
            },
            {
                "role": "user",
                "content": user_question,
            }
        ],
        model="llama3-8b-8192",
    )
    print('chat_completetion ', chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

def get_address_context(transcript_file):
    with open(transcript_file, "r") as file:
        transcript = file.read()
    user_question = "List all the instances where the speaker mentions any address or related word to emphasize any address in the audio files. Provide the address and the context of the mention. Always provide a few words before and after the address to give context of what the emergency might be. Provide exact words it speaks. Give it in a json format with address and context as keys and the values as the address and context.R Return only the array of json object"
    address_context = transcript_chat_completion(client, transcript, user_question)
    print('address context ', address_context)  
    return address_context





def get_address_context_directory(input_dir, output_dir):
    """
    Get the address context for all the transcriptions in a directory and save the results in a CSV file.

    Parameters:
    - input_dir: Path to the directory containing the input transcription files.
    - output_dir: Path to the directory where the output CSV file will be saved.
    """

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    output_file = os.path.join(output_dir, "addresses_with_context.txt")
    # Loop through all files in the input directory
    for file_name in os.listdir(input_dir):
        if not file_name.endswith(".txt"):
            continue
        # Construct the input and output file paths
        input_file = os.path.join(input_dir, file_name)
        
        time.sleep(3)
        # Get the address context for the transcription file
        address_context = get_address_context(input_file)

        #address context is a json string, store it in a new line in the .txt file 
        with open(output_file, 'a') as file:
            file.write(address_context + "\n")
        













In [200]:
input_directory = './Dallas County/Sachse Police and Fire Aug 13'
output_directory = './Dallas County/Sachse Police and Fire Aug 13'

get_address_context_directory(input_directory, output_directory)

chat_completetion  Here is the result:

[
  {"address":"to the PD", "context":"for report"},
  {"address":"10-1", "context":"if you can go"}
]
address context  Here is the result:

[
  {"address":"to the PD", "context":"for report"},
  {"address":"10-1", "context":"if you can go"}
]
chat_completetion  Here is the answer:

[
  {"address": "", "context": "295 back to PD"}
]

Please note that there is no specific address mentioned in the given audio files, only a radio channel number "295" and a code "PD" likely referring to a Police Department.
address context  Here is the answer:

[
  {"address": "", "context": "295 back to PD"}
]

Please note that there is no specific address mentioned in the given audio files, only a radio channel number "295" and a code "PD" likely referring to a Police Department.
chat_completetion  Here is the list of instances where the speaker mentions any address or related word:

[
  {"address": "78th", "context": "I've got to be checking from 78th and 5th back